In [ ]:
# Bloque 1: Importaciones y docstring inicial

"""
Temporal Fusion Transformer Optimizado
-----------------------------------------

Esta versión implementa las optimizaciones generales propuestas en
el código original publicado en 2020. Se ha estructurado el código para mejorar su
legibilidad, eficiencia computacional y mantenibilidad, sin alterar la lógica esencial
del modelo.

Modificaciones destacadas:
- Normalización de argumentos opcionales al inicio de __init__
- Inicialización optimizada de estados LSTM usando unsqueeze/expand
- Opción para compilar el método forward con torch.compile (PyTorch 2.0+)
- Eliminación de código comentado redundante
- Documentación detallada (docstrings) para cambios y funciones clave
- NUEVAS OPTIMIZACIONES: Cacheo de máscaras, fusión de operaciones, 
  vectorización y funciones JIT para mejorar rendimiento
"""

import math
from copy import copy
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import torch
from torch import nn
from torchmetrics import Metric as LightningMetric

from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.metrics import (
    MAE,
    MAPE,
    RMSE,
    SMAPE,
    MultiHorizonMetric,
    QuantileLoss,
)
from pytorch_forecasting.models.base_model import BaseModelWithCovariates
from pytorch_forecasting.models.nn import LSTM, MultiEmbedding
from pytorch_forecasting.models.temporal_fusion_transformer.sub_modules import (
    AddNorm,
    GateAddNorm,
    GatedLinearUnit,
    GatedResidualNetwork,
    InterpretableMultiHeadAttention,
    VariableSelectionNetwork,
)
from pytorch_forecasting.utils import (
    create_mask,
    detach,
    integer_histogram,
    masked_op,
    padded_stack,
    to_list,
)
from pytorch_forecasting.utils._dependencies import _check_matplotlib

In [ ]:
# Bloque 2: Definición de la clase principal y funciones auxiliares optimizadas

class TemporalFusionTransformer(BaseModelWithCovariates):
    # Inicialización optimizada de estados LSTM
    @staticmethod
    @torch.jit.script
    def initialize_lstm_states(hidden, cell, layers: int):
        """
        Inicializa los estados LSTM de manera optimizada con JIT.
        
        Args:
            hidden: Estado hidden inicial.
            cell: Estado cell inicial.
            layers: Número de capas LSTM.
            
        Returns:
            Tupla de estados inicializados (hidden, cell).
        """
        return (
            hidden.unsqueeze(0).expand(layers, -1, -1),
            cell.unsqueeze(0).expand(layers, -1, -1)
        )

    # Fusión de operaciones en el procesamiento LSTM
    def process_lstm_output(self, lstm_output, residual_input, is_encoder=True):
        """
        Combina las operaciones de gate y add_norm para el procesamiento post-LSTM.
        
        Args:
            lstm_output: Salida del LSTM.
            residual_input: Entrada residual para la conexión skip.
            is_encoder: Si se procesa el encoder (True) o decoder (False).
            
        Returns:
            Salida procesada con gate y add_norm.
        """
        gate = self.post_lstm_gate_encoder if is_encoder else self.post_lstm_gate_decoder
        add_norm = self.post_lstm_add_norm_encoder if is_encoder else self.post_lstm_add_norm_decoder
        return add_norm(gate(lstm_output), residual_input)

    # Atención eficiente en memoria
    def efficient_attention(self, q, k, v, mask=None):
        """
        Implementación de atención eficiente en memoria para secuencias largas.
        
        Args:
            q: Queries.
            k: Keys.
            v: Values.
            mask: Máscara para la atención.
            
        Returns:
            Tupla de (salida de atención, pesos de atención).
        """
        batch_size, seq_len, d_model = q.shape
        head_dim = d_model // self.hparams.attention_head_size
        
        # Reshape para computación por cabezas
        q = q.view(batch_size, seq_len, self.hparams.attention_head_size, head_dim)
        k = k.view(batch_size, -1, self.hparams.attention_head_size, head_dim)
        v = v.view(batch_size, -1, self.hparams.attention_head_size, head_dim)
        
        # Calcular atención eficientemente usando operaciones por lotes
        scores = torch.einsum("bqhd,bkhd->bhqk", q, k) / math.sqrt(head_dim)
        
        if mask is not None:
            scores = scores.masked_fill(~mask.unsqueeze(1), -1e9)
            
        attn_weights = torch.softmax(scores, dim=-1)
        context = torch.einsum("bhqk,bkhd->bqhd", attn_weights, v)
        
        return context.reshape(batch_size, seq_len, d_model), attn_weights

    # Vectorización del procesamiento de interpretación
    def process_variables_importance(self, variables, lengths):
        """
        Vectoriza el procesamiento de importancia de variables.
        
        Args:
            variables: Variables a procesar.
            lengths: Longitudes efectivas.
            
        Returns:
            Importancia de variables procesada.
        """
        mask = create_mask(variables.size(1), lengths).unsqueeze(-1)
        masked_vars = variables.masked_fill(mask, 0.0).sum(dim=1)
        return masked_vars / lengths.clamp_min(1).unsqueeze(-1)

    # Procesamiento paralelo para salida multiobjetivo
    def transform_multi_output(self, output):
        """
        Procesa todos los outputs en paralelo en lugar de secuencialmente.
        
        Args:
            output: Salida de la red.
            
        Returns:
            Lista de outputs procesados para cada target.
        """
        if self.n_targets > 1:
            # Procesar todos los outputs en paralelo en lugar de secuencialmente
            stacked_out = torch.stack([ol.weight for ol in self.output_layer])
            stacked_bias = torch.stack([ol.bias for ol in self.output_layer])
            
            # Reshape para permitir multiplicación matricial en batch
            reshaped_out = output.unsqueeze(1)  # [batch, 1, hidden]
            transformed = torch.bmm(
                reshaped_out.expand(-1, self.n_targets, -1),  # [batch, n_targets, hidden]
                stacked_out.transpose(1, 2)  # [n_targets, hidden, output_size]
            )
            transformed = transformed + stacked_bias.unsqueeze(0)
            return [transformed[:, i] for i in range(self.n_targets)]
        else:
            return self.output_layer(output)

In [ ]:
# Bloque 3: Método de inicialización (init)

    def __init__(
        self,
        hidden_size: int = 16,
        lstm_layers: int = 1,
        dropout: float = 0.1,
        output_size: Union[int, List[int]] = 7,
        loss: MultiHorizonMetric = None,
        attention_head_size: int = 4,
        max_encoder_length: int = 10,
        static_categoricals: Optional[List[str]] = None,
        static_reals: Optional[List[str]] = None,
        time_varying_categoricals_encoder: Optional[List[str]] = None,
        time_varying_categoricals_decoder: Optional[List[str]] = None,
        categorical_groups: Optional[Union[Dict, List[str]]] = None,
        time_varying_reals_encoder: Optional[List[str]] = None,
        time_varying_reals_decoder: Optional[List[str]] = None,
        x_reals: Optional[List[str]] = None,
        x_categoricals: Optional[List[str]] = None,
        hidden_continuous_size: int = 8,
        hidden_continuous_sizes: Optional[Dict[str, int]] = None,
        embedding_sizes: Optional[Dict[str, Tuple[int, int]]] = None,
        embedding_paddings: Optional[List[str]] = None,
        embedding_labels: Optional[Dict[str, np.ndarray]] = None,
        learning_rate: float = 1e-3,
        log_interval: Union[int, float] = -1,
        log_val_interval: Union[int, float] = None,
        log_gradient_flow: bool = False,
        reduce_on_plateau_patience: int = 1000,
        monotone_constaints: Optional[Dict[str, int]] = None,
        share_single_variable_networks: bool = False,
        causal_attention: bool = True,
        logging_metrics: Optional[nn.ModuleList] = None,
        use_compile: bool = False,
        **kwargs,
    ):
        """
        Temporal Fusion Transformer para series temporales.
        
        Se han aplicado optimizaciones en el manejo de argumentos opcionales
        y se añade la posibilidad de compilar el método forward para mejorar
        el rendimiento en PyTorch 2.0+.

        Args:
            hidden_size: Tamaño de la capa oculta.
            lstm_layers: Número de capas LSTM.
            dropout: Tasa de dropout.
            output_size: Número de salidas (p.ej.: número de cuantiles en QuantileLoss).
            loss: Función de pérdida (debe ser un LightningMetric).
            attention_head_size: Número de cabezas en la atención.
            max_encoder_length: Longitud máxima del encoder.
            static_categoricals: Lista de variables categóricas estáticas.
            static_reals: Lista de variables continuas estáticas.
            time_varying_categoricals_encoder: Lista de variables categóricas para el encoder.
            time_varying_categoricals_decoder: Lista de variables categóricas para el decoder.
            categorical_groups: Diccionario o lista de grupos de variables categóricas.
            time_varying_reals_encoder: Lista de variables continuas para el encoder.
            time_varying_reals_decoder: Lista de variables continuas para el decoder.
            x_reals: Orden de variables continuas en el tensor de entrada.
            x_categoricals: Orden de variables categóricas en el tensor de entrada.
            hidden_continuous_size: Tamaño oculto para variables continuas.
            hidden_continuous_sizes: Diccionario que mapea variables continuas a tamaños específicos.
            embedding_sizes: Diccionario que mapea nombres de variables categóricas a tuplas (número de clases, tamaño del embedding).
            embedding_paddings: Lista de variables categóricas con padding.
            embedding_labels: Diccionario que mapea nombres de variables categóricas a etiquetas.
            learning_rate: Tasa de aprendizaje.
            log_interval: Intervalo para logging de predicciones.
            log_val_interval: Intervalo para logging en validación.
            log_gradient_flow: Si se debe loggear el flujo de gradientes.
            reduce_on_plateau_patience: Paciencia para reducir la tasa de aprendizaje.
            monotone_constaints: Restricciones de monotonía para variables continuas.
            share_single_variable_networks: Si compartir la red de variable única entre encoder y decoder.
            causal_attention: Si se aplica atención causal en el decoder.
            logging_metrics: Lista de métricas a loggear durante el entrenamiento.
            use_compile: Si se debe compilar el método forward (requiere PyTorch 2.0+).
            **kwargs: Argumentos adicionales para BaseModel.
        """
        # Normalización de argumentos opcionales para evitar múltiples comprobaciones posteriores
        static_categoricals = static_categoricals or []
        static_reals = static_reals or []
        time_varying_categoricals_encoder = time_varying_categoricals_encoder or []
        time_varying_categoricals_decoder = time_varying_categoricals_decoder or []
        time_varying_reals_encoder = time_varying_reals_encoder or []
        time_varying_reals_decoder = time_varying_reals_decoder or []
        x_categoricals = x_categoricals or []
        x_reals = x_reals or []
        embedding_labels = embedding_labels or {}
        embedding_paddings = embedding_paddings or []
        embedding_sizes = embedding_sizes or {}
        hidden_continuous_sizes = hidden_continuous_sizes or {}
        categorical_groups = categorical_groups or {}
        if monotone_constaints is None:
            monotone_constaints = {}
        if logging_metrics is None:
            logging_metrics = nn.ModuleList([SMAPE(), MAE(), RMSE(), MAPE()])
        if loss is None:
            loss = QuantileLoss()

        # Se guardan los hiperparámetros (incluyendo los que ya tienen valor por defecto)
        super().__init__(loss=loss, logging_metrics=logging_metrics, **kwargs)
        self.save_hyperparameters(ignore=["use_compile"])
        self.hparams.use_compile = use_compile  # almacenar flag de compilación

        # Creación de los módulos de procesamiento de inputs
        # 1. Embeddings para variables categóricas
        self.input_embeddings = MultiEmbedding(
            embedding_sizes=self.hparams.embedding_sizes,
            categorical_groups=self.hparams.categorical_groups,
            embedding_paddings=self.hparams.embedding_paddings,
            x_categoricals=self.hparams.x_categoricals,
            max_embedding_size=self.hparams.hidden_size,
        )

        # 2. Procesamiento de variables continuas a través de capas lineales (prescalers)
        self.prescalers = nn.ModuleDict(
            {
                name: nn.Linear(
                    1,
                    self.hparams.hidden_continuous_sizes.get(
                        name, self.hparams.hidden_continuous_size
                    ),
                )
                for name in self.reals  # se asume que self.reals se define en la clase base
            }
        )

        # 3. Variable Selection para variables estáticas, encoder y decoder
        # Variables estáticas
        static_input_sizes = {
            name: self.input_embeddings.output_size[name]
            for name in self.hparams.static_categoricals
        }
        static_input_sizes.update(
            {
                name: self.hparams.hidden_continuous_sizes.get(
                    name, self.hparams.hidden_continuous_size
                )
                for name in self.hparams.static_reals
            }
        )
        self.static_variable_selection = VariableSelectionNetwork(
            input_sizes=static_input_sizes,
            hidden_size=self.hparams.hidden_size,
            input_embedding_flags={
                name: True for name in self.hparams.static_categoricals
            },
            dropout=self.hparams.dropout,
            prescalers=self.prescalers,
        )

        # Variables para encoder y decoder
        encoder_input_sizes = {
            name: self.input_embeddings.output_size[name]
            for name in self.hparams.time_varying_categoricals_encoder
        }
        encoder_input_sizes.update(
            {
                name: self.hparams.hidden_continuous_sizes.get(
                    name, self.hparams.hidden_continuous_size
                )
                for name in self.hparams.time_varying_reals_encoder
            }
        )
        decoder_input_sizes = {
            name: self.input_embeddings.output_size[name]
            for name in self.hparams.time_varying_categoricals_decoder
        }
        decoder_input_sizes.update(
            {
                name: self.hparams.hidden_continuous_sizes.get(
                    name, self.hparams.hidden_continuous_size
                )
                for name in self.hparams.time_varying_reals_decoder
            }
        )

        # Opción de compartir redes de variable única entre encoder y decoder
        if self.hparams.share_single_variable_networks:
            self.shared_single_variable_grns = nn.ModuleDict()
            for name, input_size in encoder_input_sizes.items():
                self.shared_single_variable_grns[name] = GatedResidualNetwork(
                    input_size,
                    min(input_size, self.hparams.hidden_size),
                    self.hparams.hidden_size,
                    self.hparams.dropout,
                )
            for name, input_size in decoder_input_sizes.items():
                if name not in self.shared_single_variable_grns:
                    self.shared_single_variable_grns[name] = GatedResidualNetwork(
                        input_size,
                        min(input_size, self.hparams.hidden_size),
                        self.hparams.hidden_size,
                        self.hparams.dropout,
                    )

        self.encoder_variable_selection = VariableSelectionNetwork(
            input_sizes=encoder_input_sizes,
            hidden_size=self.hparams.hidden_size,
            input_embedding_flags={
                name: True for name in self.hparams.time_varying_categoricals_encoder
            },
            dropout=self.hparams.dropout,
            context_size=self.hparams.hidden_size,
            prescalers=self.prescalers,
            single_variable_grns=(
                {} if not self.hparams.share_single_variable_networks else self.shared_single_variable_grns
            ),
        )

        self.decoder_variable_selection = VariableSelectionNetwork(
            input_sizes=decoder_input_sizes,
            hidden_size=self.hparams.hidden_size,
            input_embedding_flags={
                name: True for name in self.hparams.time_varying_categoricals_decoder
            },
            dropout=self.hparams.dropout,
            context_size=self.hparams.hidden_size,
            prescalers=self.prescalers,
            single_variable_grns=(
                {} if not self.hparams.share_single_variable_networks else self.shared_single_variable_grns
            ),
        )

        # 4. Codificadores estáticos para variable selection, estado inicial y enriquecimiento
        self.static_context_variable_selection = GatedResidualNetwork(
            input_size=self.hparams.hidden_size,
            hidden_size=self.hparams.hidden_size,
            output_size=self.hparams.hidden_size,
            dropout=self.hparams.dropout,
        )
        self.static_context_initial_hidden_lstm = GatedResidualNetwork(
            input_size=self.hparams.hidden_size,
            hidden_size=self.hparams.hidden_size,
            output_size=self.hparams.hidden_size,
            dropout=self.hparams.dropout,
        )
        self.static_context_initial_cell_lstm = GatedResidualNetwork(
            input_size=self.hparams.hidden_size,
            hidden_size=self.hparams.hidden_size,
            output_size=self.hparams.hidden_size,
            dropout=self.hparams.dropout,
        )
        self.static_context_enrichment = GatedResidualNetwork(
            self.hparams.hidden_size,
            self.hparams.hidden_size,
            self.hparams.hidden_size,
            self.hparams.dropout,
        )

        # 5. LSTM para procesamiento local: encoder (histórico) y decoder (futuro)
        self.lstm_encoder = LSTM(
            input_size=self.hparams.hidden_size,
            hidden_size=self.hparams.hidden_size,
            num_layers=self.hparams.lstm_layers,
            dropout=self.hparams.dropout if self.hparams.lstm_layers > 1 else 0,
            batch_first=True,
        )
        self.lstm_decoder = LSTM(
            input_size=self.hparams.hidden_size,
            hidden_size=self.hparams.hidden_size,
            num_layers=self.hparams.lstm_layers,
            dropout=self.hparams.dropout if self.hparams.lstm_layers > 1 else 0,
            batch_first=True,
        )

        # 6. Skip connections y normalización post-LSTM
        self.post_lstm_gate_encoder = GatedLinearUnit(
            self.hparams.hidden_size, dropout=self.hparams.dropout
        )
        # Reutilizamos la misma instancia para decoder para simplificar
        self.post_lstm_gate_decoder = self.post_lstm_gate_encoder
        self.post_lstm_add_norm_encoder = AddNorm(
            self.hparams.hidden_size, trainable_add=False
        )
        self.post_lstm_add_norm_decoder = self.post_lstm_add_norm_encoder

        # 7. Enriquecimiento estático y atención para procesamiento a largo alcance
        self.static_enrichment = GatedResidualNetwork(
            input_size=self.hparams.hidden_size,
            hidden_size=self.hparams.hidden_size,
            output_size=self.hparams.hidden_size,
            dropout=self.hparams.dropout,
            context_size=self.hparams.hidden_size,
        )
        self.multihead_attn = InterpretableMultiHeadAttention(
            d_model=self.hparams.hidden_size,
            n_head=self.hparams.attention_head_size,
            dropout=self.hparams.dropout,
        )
        self.post_attn_gate_norm = GateAddNorm(
            self.hparams.hidden_size, dropout=self.hparams.dropout, trainable_add=False
        )
        self.pos_wise_ff = GatedResidualNetwork(
            self.hparams.hidden_size,
            self.hparams.hidden_size,
            self.hparams.hidden_size,
            dropout=self.hparams.dropout,
        )

        # 8. Procesamiento de salida (sin dropout en esta etapa)
        self.pre_output_gate_norm = GateAddNorm(
            self.hparams.hidden_size, dropout=None, trainable_add=False
        )
        if self.n_targets > 1:  # arquitectura multiobjetivo
            self.output_layer = nn.ModuleList(
                [
                    nn.Linear(self.hparams.hidden_size, osize)
                    for osize in (self.hparams.output_size if isinstance(self.hparams.output_size, list)
                                  else [self.hparams.output_size])
                ]
            )
        else:
            self.output_layer = nn.Linear(
                self.hparams.hidden_size, self.hparams.output_size
            )

        # Cache para máscaras de atención
        self._attention_mask_cache = {}
        
        # Compilación mejorada para otras funciones críticas
        if self.hparams.use_compile and hasattr(torch, "compile"):
            self.forward = torch.compile(self.forward)
            self.interpret_output = torch.compile(
                self.interpret_output,
                dynamic=True  # Para manejar tamaños variables
            )
            # Se añade un docstring indicando que el método forward ha sido compilado
            self.forward.__doc__ = (self.forward.__doc__ or "") + "\n\nOptimized with torch.compile."

In [ ]:
    # Bloque 4: Métodos de utilidad

    @classmethod
    def from_dataset(
        cls,
        dataset: TimeSeriesDataSet,
        allowed_encoder_known_variable_names: List[str] = None,
        **kwargs,
    ):
        """
        Crea el modelo a partir de un dataset.

        Args:
            dataset: Dataset de series temporales.
            allowed_encoder_known_variable_names: Lista de variables conocidas permitidas en el encoder.
            **kwargs: Argumentos adicionales (p.ej. hiperparámetros).

        Returns:
            Instancia de TemporalFusionTransformer.
        """
        new_kwargs = copy(kwargs)
        new_kwargs["max_encoder_length"] = dataset.max_encoder_length
        new_kwargs.update(
            cls.deduce_default_output_parameters(dataset, kwargs, QuantileLoss())
        )
        return super().from_dataset(
            dataset,
            allowed_encoder_known_variable_names=allowed_encoder_known_variable_names,
            **new_kwargs,
        )

    def expand_static_context(self, context: torch.Tensor, timesteps: int) -> torch.Tensor:
        """
        Expande el contexto estático a dimensión temporal.

        Args:
            context: Tensor con el contexto estático (batch, features).
            timesteps: Número de timesteps a expandir.

        Returns:
            Tensor expandido de dimensiones (batch, timesteps, features).
        """
        return context.unsqueeze(1).expand(-1, timesteps, -1)

    # Cache de máscaras de atención
    def get_attention_mask(
        self, encoder_lengths: torch.LongTensor, decoder_lengths: torch.LongTensor
    ) -> torch.Tensor:
        """
        Genera una máscara causal para la capa de self-atención con cache.

        Args:
            encoder_lengths: Longitudes efectivas del encoder en el batch.
            decoder_lengths: Longitudes efectivas del decoder en el batch.

        Returns:
            Tensor máscara combinado para el encoder y decoder.
        """
        # Usar una clave de cache basada en las dimensiones
        cache_key = (encoder_lengths.max().item(), decoder_lengths.max().item())
        if cache_key not in self._attention_mask_cache:
            decoder_length = decoder_lengths.max()
            if self.hparams.causal_attention:
                attend_step = torch.arange(decoder_length, device=self.device)
                predict_step = torch.arange(decoder_length, device=self.device).unsqueeze(1)
                decoder_mask = (attend_step >= predict_step).unsqueeze(0).expand(encoder_lengths.size(0), -1, -1)
            else:
                decoder_mask = create_mask(decoder_length, decoder_lengths).unsqueeze(1).expand(-1, decoder_length, -1)
            encoder_mask = create_mask(encoder_lengths.max(), encoder_lengths).unsqueeze(1).expand(-1, decoder_length, -1)
            self._attention_mask_cache[cache_key] = torch.cat((encoder_mask, decoder_mask), dim=2)
        return self._attention_mask_cache[cache_key]

In [ ]:
    def forward(self, x: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        """
        Ejecuta la propagación hacia adelante del modelo.

        Las dimensiones de entrada se esperan como: (n_samples x time x variables).

        Args:
            x: Diccionario con tensores de entrada, incluyendo:
                - "encoder_cat", "decoder_cat"
                - "encoder_cont", "decoder_cont"
                - "encoder_lengths", "decoder_lengths"
                - "target_scale", entre otros

        Returns:
            Diccionario con la predicción y datos auxiliares (e.g., pesos de atención).
        """
        # MEJORA 8: Precalcular dimensiones frecuentes
        batch_size = x["encoder_lengths"].size(0)
        encoder_lengths = x["encoder_lengths"]
        decoder_lengths = x["decoder_lengths"]
        max_encoder_length = int(encoder_lengths.max())
        
        # Concatenar inputs de categorías y continuos en la dimensión temporal
        x_cat = torch.cat([x["encoder_cat"], x["decoder_cat"]], dim=1)
        x_cont = torch.cat([x["encoder_cont"], x["decoder_cont"]], dim=1)
        timesteps = x_cont.size(1)
        
        input_vectors = self.input_embeddings(x_cat)
        # Agregar variables continuas a partir del orden definido en x_reals
        input_vectors.update(
            {
                name: x_cont[..., idx].unsqueeze(-1)
                for idx, name in enumerate(self.hparams.x_reals)
                if name in self.reals
            }
        )

        # Variable selection para variables estáticas
        if len(self.static_variables) > 0:
            static_embedding = {name: input_vectors[name][:, 0] for name in self.static_variables}
            static_embedding, static_variable_selection = self.static_variable_selection(static_embedding)
        else:
            static_embedding = torch.zeros(
                (batch_size, self.hparams.hidden_size),
                dtype=self.dtype, device=self.device
            )
            static_variable_selection = torch.zeros(
                (batch_size, 0), dtype=self.dtype, device=self.device
            )

        static_context_variable_selection = self.expand_static_context(
            self.static_context_variable_selection(static_embedding), timesteps
        )

        # Variable selection para encoder y decoder (a partir de embeddings)
        embeddings_varying_encoder = {
            name: input_vectors[name][:, :max_encoder_length]
            for name in self.encoder_variables
        }
        embeddings_varying_encoder, encoder_sparse_weights = self.encoder_variable_selection(
            embeddings_varying_encoder,
            static_context_variable_selection[:, :max_encoder_length],
        )
        embeddings_varying_decoder = {
            name: input_vectors[name][:, max_encoder_length:]
            for name in self.decoder_variables
        }
        embeddings_varying_decoder, decoder_sparse_weights = self.decoder_variable_selection(
            embeddings_varying_decoder,
            static_context_variable_selection[:, max_encoder_length:],
        )

        # MEJORA 3: Inicialización optimizada de estados LSTM
        init_hidden = self.static_context_initial_hidden_lstm(static_embedding)
        init_cell = self.static_context_initial_cell_lstm(static_embedding)
        input_hidden = init_hidden.unsqueeze(0).expand(self.hparams.lstm_layers, -1, -1)
        input_cell = init_cell.unsqueeze(0).expand(self.hparams.lstm_layers, -1, -1)

        # Procesamiento LSTM para encoder y decoder
        encoder_output, (hidden, cell) = self.lstm_encoder(
            embeddings_varying_encoder,
            (input_hidden, input_cell),
            lengths=encoder_lengths,
            enforce_sorted=False,
        )
        decoder_output, _ = self.lstm_decoder(
            embeddings_varying_decoder,
            (hidden, cell),
            lengths=decoder_lengths,
            enforce_sorted=False,
        )

        # MEJORA 2: Procesamiento fusionado post-LSTM para mejorar eficiencia
        lstm_output_encoder = self.process_lstm_output(encoder_output, embeddings_varying_encoder, True)
        lstm_output_decoder = self.process_lstm_output(decoder_output, embeddings_varying_decoder, False)
        lstm_output = torch.cat([lstm_output_encoder, lstm_output_decoder], dim=1)

        # Enriquecimiento estático
        static_context_enrichment = self.static_context_enrichment(static_embedding)
        attn_input = self.static_enrichment(
            lstm_output,
            self.expand_static_context(static_context_enrichment, timesteps),
        )

        # MEJORA 7: Usar atención eficiente en memoria para secuencias largas
        if hasattr(self, "efficient_attention") and max_encoder_length > 100:
            attn_output, attn_output_weights = self.efficient_attention(
                q=attn_input[:, max_encoder_length:],
                k=attn_input,
                v=attn_input,
                mask=self.get_attention_mask(encoder_lengths=encoder_lengths, decoder_lengths=decoder_lengths),
            )
        else:
            # Atención multi-cabeza con máscara (caso estándar)
            attn_output, attn_output_weights = self.multihead_attn(
                q=attn_input[:, max_encoder_length:],
                k=attn_input,
                v=attn_input,
                mask=self.get_attention_mask(encoder_lengths=encoder_lengths, decoder_lengths=decoder_lengths),
            )

        attn_output = self.post_attn_gate_norm(attn_output, attn_input[:, max_encoder_length:])
        output = self.pos_wise_ff(attn_output)

        # Skip connection final antes de salida
        output = self.pre_output_gate_norm(output, lstm_output[:, max_encoder_length:])
        
        # MEJORA 5: Procesamiento paralelo para salida multiobjetivo
        if self.n_targets > 1:
            output = self.transform_multi_output(output)
        else:
            output = self.output_layer(output)

        return self.to_network_output(
            prediction=self.transform_output(output, target_scale=x["target_scale"]),
            encoder_attention=attn_output_weights[..., :max_encoder_length],
            decoder_attention=attn_output_weights[..., max_encoder_length:],
            static_variables=static_variable_selection,
            encoder_variables=encoder_sparse_weights,
            decoder_variables=decoder_sparse_weights,
            decoder_lengths=decoder_lengths,
            encoder_lengths=encoder_lengths,
        )